異体字のペアを作成する

In [1]:
%cd itaiji

/home/analysis01/src/kanjikana-model/dict/itaiji


/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


ペアにして、属性をつける。 

key1,key2:[attr1,attr2]
key2,key1:[attr1,attr2]      

In [2]:
pair={}

In [3]:
def make_pair(fname,attr):
    with open(fname,'r',encoding='utf-8') as f:
        for l in f:
            items=l.rstrip().split(',')
            keyA=f'{items[0]},{items[1]}'      
            keyB=f'{items[1]},{items[0]}'
            if keyA not in pair:
                pair[keyA]=[]
            if keyB not in pair:
                pair[keyB]=[]
            pair[keyA].append(attr)
            alst=pair[keyA]
            pair[keyB].append(attr)
            blst=pair[keyB]
            pair[keyA]=list(set(alst))
            pair[keyB]=list(set(blst))

In [4]:
make_pair('../oss/mozc/itaiji.txt','mozc_itaiji')

In [5]:
len(pair)

1420

In [6]:
make_pair('../oss/skk/itaiji.txt','skk_itaiji')

In [7]:
len(pair)

2344

In [8]:
pair

{'叱,𠮟': ['mozc_itaiji'],
 '𠮟,叱': ['mozc_itaiji'],
 '埓,埒': ['skk_itaiji', 'mozc_itaiji'],
 '埒,埓': ['skk_itaiji', 'mozc_itaiji'],
 '髙,高': ['mozc_itaiji'],
 '高,髙': ['mozc_itaiji'],
 '啮,噛': ['mozc_itaiji'],
 '噛,啮': ['mozc_itaiji'],
 '噓,嘘': ['mozc_itaiji'],
 '嘘,噓': ['mozc_itaiji'],
 '蟬,蝉': ['mozc_itaiji'],
 '蝉,蟬': ['mozc_itaiji'],
 '驒,騨': ['mozc_itaiji'],
 '騨,驒': ['mozc_itaiji'],
 '簞,箪': ['mozc_itaiji'],
 '箪,簞': ['mozc_itaiji'],
 '剝,剥': ['mozc_itaiji'],
 '剥,剝': ['mozc_itaiji'],
 '屛,屏': ['mozc_itaiji'],
 '屏,屛': ['mozc_itaiji'],
 '穎,頴': ['skk_itaiji', 'mozc_itaiji'],
 '頴,穎': ['skk_itaiji', 'mozc_itaiji'],
 '禰,祢': ['skk_itaiji', 'mozc_itaiji'],
 '祢,禰': ['skk_itaiji', 'mozc_itaiji'],
 '儘,侭': ['skk_itaiji', 'mozc_itaiji'],
 '侭,儘': ['skk_itaiji', 'mozc_itaiji'],
 '壺,壷': ['skk_itaiji', 'mozc_itaiji'],
 '壷,壺': ['skk_itaiji', 'mozc_itaiji'],
 '彎,弯': ['skk_itaiji', 'mozc_itaiji'],
 '弯,彎': ['skk_itaiji', 'mozc_itaiji'],
 '攪,撹': ['skk_itaiji', 'mozc_itaiji'],
 '撹,攪': ['skk_itaiji', 'mozc_itaiji'],
 '檜,

In [9]:
siz={}
for k,v in pair.items():
    if len(v) not in siz:
        siz[len(v)]=0
    siz[len(v)]+=1

In [10]:
siz

{1: 1112, 2: 1232}

同じ文字のグループを作成する

In [11]:
groups=[]
for k,v in pair.items():
    key1,key2=k.split(',')
    for i in range(len(groups)):
        if key1 in groups[i]:
            groups[i].append(key2)
            groups[i]=list(set(groups[i]))
            break
        elif key2 in groups[i]:
            groups[i].append(key1)
            groups[i]=list(set(groups[i]))
            break
    else:
        groups.append([key1,key2])


In [12]:
groups

[['叱', '𠮟'],
 ['埓', '埒'],
 ['高', '髙'],
 ['啮', '嚙', '噛'],
 ['噓', '嘘'],
 ['蝉', '蟬'],
 ['驒', '騨'],
 ['箪', '簞'],
 ['剝', '剥'],
 ['屛', '屏'],
 ['頴', '穎'],
 ['禰', '祢', '袮'],
 ['儘', '侭'],
 ['壷', '壺'],
 ['灣', '湾', '弯', '彎'],
 ['撹', '攪'],
 ['桧', '檜'],
 ['梼', '檮'],
 ['檪', '櫟'],
 ['涛', '濤'],
 ['灌', '濯', '潅'],
 ['瀘', '濾', '沪'],
 ['箏', '筝'],
 ['篭', '籠'],
 ['籔', '薮', '藪'],
 ['蘆', '芦'],
 ['蛎', '蠣'],
 ['蝿', '蠅'],
 ['諌', '諫'],
 ['賎', '賤'],
 ['迩', '邇'],
 ['靱', '靭', '靫'],
 ['頸', '頚'],
 ['鯵', '鰺'],
 ['鶯', '鴬'],
 ['侠', '俠'],
 ['唖', '啞'],
 ['囊', '嚢'],
 ['塡', '填'],
 ['掻', '搔'],
 ['發', '醗', '潑', '発', '溌'],
 ['涜', '瀆'],
 ['焔', '焰'],
 ['禱', '祷'],
 ['繡', '繍', '綉'],
 ['繫', '繋'],
 ['萊', '莱'],
 ['蒋', '蔣'],
 ['蠟', '蝋'],
 ['軀', '躯'],
 ['醤', '醬'],
 ['醱', '發', '醗'],
 ['頬', '頰'],
 ['顚', '顛'],
 ['鴎', '鷗'],
 ['鹼', '鹸'],
 ['麹', '麴'],
 ['緣', '縁'],
 ['横', '橫'],
 ['黃', '黄'],
 ['温', '溫'],
 ['概', '槪'],
 ['渴', '渇'],
 ['寬', '寛'],
 ['虚', '虛'],
 ['郷', '鄕'],
 ['薫', '薰', '熏', '燻'],
 ['掲', '揭'],
 ['研', '硏'],
 ['黒', '黑'],
 ['緖', '緒'],
 [

In [13]:
len(groups)

771

第一水準から順に優先順をつけて、優先のものに代替するようにする
さらに小中高の順に優先する

In [14]:
def load_jis(fname):
    with open(fname,'r',encoding='utf-8') as f:
        hsh ={l.rstrip():1 for l in f if len(l.rstrip())>0}
    return hsh

In [15]:
dai1=load_jis('../daitai/jisdai1.dat')

In [16]:
dai2=load_jis('../daitai/jisdai2.dat')

In [17]:
dai3=load_jis('../daitai/jisdai3.dat')

In [18]:
dai4=load_jis('../daitai/jisdai4.dat')

In [19]:
syo=load_jis('../daitai/syogaku.dat')

In [20]:
cyu=load_jis('../daitai/cyugaku.dat')

In [21]:
kou=load_jis('../daitai/koukou.dat')

In [22]:
suijun=[syo,cyu,kou,dai1,dai2,dai3,dai4]

In [23]:
import regex
import re

In [24]:
def get_suijun(s):
    # 平仮名カタカナは最優先 0でかえす  旧字は除外
    if (regex.match('^\p{Script=Hiragana}+$',s) is not None or regex.match('^\p{Script=Katakana}+$',s) is not None) and re.match(f'^[ゐゑ	゚゛゜ゝゞゟ]',s) is None and re.match('^[ヰヱヸヹヽヾヿ]+$',s) is None:
        return 0 
    
    for i,sui in enumerate(suijun):
        if s in sui:
            return i+1
    return len(suijun) # 1-4水準外

In [25]:
def get_min_suijun(lst): # 配列内で最も水準が高いものの水準
    min_suijun=5
    min_moji=''
    for l in lst:
        sui=get_suijun(l)
        if min_suijun>sui:
            min_suijun=sui
            min_moji=l
    if min_suijun==5:
        #min_moji=lst[0]  # 全部1ー4水準外の時は最初のものを選択
        # 最もコードポイントが小さいもの
        clst=[ord(l) for l in lst]
        min_moji=chr(min(clst))
        print(min_moji)
    return min_suijun,min_moji        

In [26]:
ngroup={}  # キーが水準の低いもののみ残す  {key2:key1,key3:key1,key4:key1}  key1 is min suijun

In [27]:
for vv in groups:
    min_sui,min_moji=get_min_suijun(vv)
    for val in vv:
        if val =="䨠" or min_moji=="䨠":
            continue
        if val == min_moji:
            continue
        key=f'{min_moji},{val}'
        attr=[]
        if key in pair:
            attr=list(set(pair[key]))
        ngroup[val]={min_moji:{"dics":attr}}


埒
屏
檪
沪
筝
匳
妍
槞
犲
秕
剳
鵙
卍
偖
冽
腮
襪
煕
并
嚏
坡
壜
佞
嫺
嫐
彿
忰
戛
揀
椁
杠
熏
犁
盪
眥
睹
瑙
箘
緕
櫑
萼
蒂
諡
譖
噪
狢
豼
獏
跖
爍
鈩
鰛
鳧
鵝
鶇
鷆
伜
崘
厠
厦
厮
彜
抬
昜
樒
檐
檗
梹
琅
甎
皸
疣
腟
舮
苺
虱
螳
蟆
蟒
蠧
衂
褝
躔
輒
逎
逅
閙
鰲
囓
懴
炯
俎
筐
嵌
籖
芻
蚋
賍
鑚
麁
麩
仞
僣
冦
冤
卮
廸
殱
珱
畴
羮
羇
韲
痲
佇
笋
籐
籥
冪
猊
啖
咯
扞
衽
轢
飃
棕
浣
薀
訛
址
拏
軅
帚
膾
瑤
劒
輌
啣
旙
晧
挧
墸
杆
銹
慙
鴟
恷
凛
攴
釶
錏
臈
奘
凭
陏
禀
靹
繿


In [28]:
len(ngroup)

933

In [29]:
ngroup

{'𠮟': {'叱': {'dics': ['mozc_itaiji']}},
 '埓': {'埒': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '髙': {'高': {'dics': ['mozc_itaiji']}},
 '啮': {'噛': {'dics': ['mozc_itaiji']}},
 '嚙': {'噛': {'dics': ['mozc_itaiji']}},
 '噓': {'嘘': {'dics': ['mozc_itaiji']}},
 '蟬': {'蝉': {'dics': ['mozc_itaiji']}},
 '驒': {'騨': {'dics': ['mozc_itaiji']}},
 '簞': {'箪': {'dics': ['mozc_itaiji']}},
 '剥': {'剝': {'dics': ['mozc_itaiji']}},
 '屛': {'屏': {'dics': ['mozc_itaiji']}},
 '穎': {'頴': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '祢': {'禰': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '袮': {'禰': {'dics': ['skk_itaiji']}},
 '儘': {'侭': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '壺': {'壷': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '灣': {'湾': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '弯': {'湾': {'dics': ['skk_itaiji']}},
 '彎': {'湾': {'dics': ['skk_itaiji']}},
 '攪': {'撹': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '檜': {'桧': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '檮': {'梼': {'dics': ['skk_itaiji', 'mozc_itaiji']}

In [30]:
import json

In [31]:
with open('../itaiji_o.json','w',encoding='utf-8') as f:
    json.dump(ngroup,f,ensure_ascii=False,indent=2)

In [32]:
!date

2025年 10月  3日 金曜日 16:03:57 JST
